In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score
import shap

Types of **SHAP** techniques (not really **model-agnostic** anymore but boosts performance of specific model):
- **KernelSHAP**
- **TreeSHAP**
- **DeepSHAP**

# SHAP
**SHapley Additive exPlanations** <br>
SHAP comes from **Cooperative Game Theory**<br> <br>
- Imagine a game with $x$ players, all of them forming a **coalition**
- When the game is over the gain a certain payout compared with their contributions
- What is a fair distribution? $\rightarrow$ **Shapley values**
![alt text](images/shap_ex.png) <br>
- Players $\rightarrow$ **Features**
- Payout $\rightarrow$ **Model Prediction**
- Game $\rightarrow$ **ML Blackbox model** <br> <br>
**The Main idea** : $\rightarrow$ How the **prediction** (payout) would perform in different **Coalition** variations. (with or without some players) (every possible subset), then 
1. We calculate each players contribution for each subset (**coalition**)
2. **Averaging** over all this contributions that gives us $\rightarrow$ **Marginal Contribution** of a player to the team <br> <br>
---
### The paper (A Unified Approach to Interpreting Model Predictions)
The Explainable AI technique **SHAP** simply makes use of these **shapley values** <br>
- In general, it is a **local explanation technique** 
- **AIM:** to explain individual predicitons of black box models
- However, it is possible to get **global explanations** through aggregating over these individual predictions<br><br>
---
### Calculating shapley values
$\phi_i \rightarrow$ Shapley value for feature $i$ **(e.g.: {Age})**<br>
$f \rightarrow$ Black Box Model<br>
$x \rightarrow$ Input data point<br>
$z' \rightarrow$ Subset **(e.g.: {Age, BMI})**<br>
$x' \rightarrow$ Simplified data input<br>
Using a **mapping function** we transform $x \rightarrow x'$ <br>
$z'\subseteq x' \rightarrow$ Iterate over all possible combinations of features<br>
$f_x(z') \rightarrow$ Black Box Model output for our input **with** the feature(s) we are interested in **(e.g.:{Age, BMI})**<br>
$f_x(z' \backslash i) \rightarrow$ Black Box Model output for our input **without** the feature(s) we are interested in **(e.g.:{BMI})**<br>
$[f_x(z') - f_x(z' \backslash i)] \rightarrow$ Tells us how feature $i$ contributed to that subset. Also called the **marginal value**<br>
$M \rightarrow$ Total number of features<br>
$\frac{|z'|!(M - |z'| - 1)!}{M!} \rightarrow$ Weighting according to how many players are in that correlation<br><br>
$$\phi_i(f,x) = \sum_{z'\subseteq x'} \frac{|z'|!(M - |z'| - 1)!}{M!} [f_x(z') - f_x(z' \backslash i)]$$
<br>
- Intuition: The contribution of adding the feature $i$ should be weighted more if already many features are included in that subset
<br>
---
### How do we exclude a feature from a ML model?
![alt text](images/shap_ex2.png)
- For the features we want to exclude we just **input random values from the train data set** and do this for all subsets
- A random feature has usually no predictive power
![alt text](images/shap_ex3.png)
<br>
---
### Complexity of calculating Shapley Values
- Where $n$ is the number of features <br>
$$2^n = \text{total number of subsets of a set}$$
- For $10$ features:
$$2^{10} = 1024$$ <br>
> The basic idea is (as presented in the SHAP paper) that we can **simply approximate** the shapley values instead of calculating all possible combinations
>> **KernelSHAP** for instance samples feature subsets and fits a linear regression model based on these samples. <br>
>> The Variables in this Linear Regression model are simply: if a feature is present or absent. <br>
>> After the training the coefficients $\beta_n$ can be interpreted as approximated **shapley values**. <br>
>> $$Y=x_1\beta_1+x_2\beta_2+...$$ <br>
>> So we weight our samples based on how much information they contain
                                                          